In [2]:
import twitter

In [3]:
api = twitter.Api(consumer_key='',
                      consumer_secret='',
                      access_token_key='',
                      access_token_secret='',
                  tweet_mode='')

In [47]:
print(api.VerifyCredentials())

{"created_at": "Wed Oct 23 13:27:35 +0000 2019", "default_profile": true, "default_profile_image": true, "description": "I want to pass explo class", "id": 1186997330458959872, "id_str": "1186997330458959872", "name": "AnnaExplorer", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ExplorerAnna"}


In [64]:
x =api.GetSearch(
    raw_query="q=%23brexit%20&result_type=recent")

In [67]:
x

[Status(ID=1192086306551140353, ScreenName=GeodanNew, Created=Wed Nov 06 14:28:18 +0000 2019, Text='RT @patcondell: The #Tories want to keep us as closely tied to the #EU as possible (while falsely claiming to have delivered #Brexit) so th…'),
 Status(ID=1192086298611388416, ScreenName=WaddyIsRight, Created=Wed Nov 06 14:28:16 +0000 2019, Text="#Brexit! #UkElection! #ImpeachmentInquiry! You name it - it's in this week's Newsmaker interview @AM1480WLEA https://t.co/ozx3AoZgua #impeachment #UKelection2019 #borisjohnson #NigelFarage #BrexitDeal #Brexitcast #Conservative #Conservatives #ConservativeParty #JeremyCorbyn #MAGA"),
 Status(ID=1192086295172067328, ScreenName=WaiKamachi, Created=Wed Nov 06 14:28:15 +0000 2019, Text='RT @thatginamiller: All concerned about UK democracy &amp; security should send a message to @BorisJohnson demanding publication of report sign…'),
 Status(ID=1192086288415051776, ScreenName=ziggyc79, Created=Wed Nov 06 14:28:13 +0000 2019, Text='RT @RachelEden: If yo

In [40]:
import nltk